In [10]:
using MLJ
using ScikitLearn
using Random, Statistics
using CSV
using DataFrames

In [11]:
# read the CSV file
df = CSV.read("hikari.csv", DataFrame)

df = select(df, [:"idle.tot", :"flow_iat.tot", :"fwd_iat.tot", :"active.avg", :"idle.max", :"flow_iat.max", :"fwd_iat.max", :"active.max", :"idle.avg", :"active.tot", :"bwd_iat.max", :"idle.min", :"flow_iat.avg", :"fwd_iat.avg", :"flow_iat.min", :"fwd_iat.min", :"fwd_iat.std", :"bwd_iat.std", :"flow_iat.std", :"bwd_iat.avg", :"Label"])



train_ratio = 0.8
test_ratio = 1.0 - train_ratio

nrows = nrow(df)
train_indices = randsubseq(1:nrows, train_ratio)
test_indices = setdiff(1:nrows, train_indices)

train_df = df[train_indices, :]
test_df = df[test_indices, :]

Row,idle.tot,flow_iat.tot,fwd_iat.tot,active.avg,idle.max,flow_iat.max,fwd_iat.max,active.max,idle.avg,active.tot,bwd_iat.max,idle.min,flow_iat.avg,fwd_iat.avg,flow_iat.min,fwd_iat.min,fwd_iat.std,bwd_iat.std,flow_iat.std,bwd_iat.avg,Label
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,0.0,2.2076e6,2.2076e6,2.2076e6,0.0,1.96376e6,1.96376e6,2.2076e6,0.0,2.2076e6,2.03293e6,0.0,78843.0,1.57686e5,7.86781,18.1198,5.20505e5,5.60627e5,3.69638e5,1.67535e5,1
2,1.19681e7,1.22034e7,1.22034e7,117619.0,1.19681e7,1.19681e7,1.19681e7,2.06325e5,1.19681e7,2.35238e5,1.20367e7,1.19681e7,4.69361e5,9.38722e5,15.0204,36.0012,3.31403e6,3.47111e6,2.34534e6,1.01457e6,1
3,1.58484e7,1.60825e7,1.60825e7,1.17077e5,1.58484e7,1.58484e7,1.58484e7,205157.0,1.58484e7,234153.0,1.59166e7,1.58484e7,5.95649e5,1.23712e6,8.10623,33.1402,4.39021e6,4.41134e6,3.04832e6,1.23482e6,1
4,1.37657e7,1.40043e7,1.40043e7,1.19272e5,1.37657e7,1.37657e7,1.37657e7,208838.0,1.37657e7,2.38545e5,1.38346e7,1.37657e7,5.38626e5,1.07725e6,6.91414,19.7887,3.8125e6,3.99004e6,2.69784e6,1.16455e6,1
5,0.0,4.88966e6,4.88966e6,4.88966e6,0.0,4.42115e6,4.42115e6,4.88966e6,0.0,4.88966e6,4.48944e6,0.0,1.88064e5,3.76128e5,10.9673,25.034,1.2184e6,1.28814e6,8.64762e5,4.04992e5,1
6,1.10432e7,1.12768e7,1.12768e7,1.16798e5,1.10432e7,1.10432e7,1.10432e7,204556.0,1.10432e7,2.33596e5,1.1112e7,1.10432e7,4.33722e5,8.67444e5,13.8283,18.1198,3.05754e6,3.20423e6,2.16395e6,9.37251e5,1
7,0.0,2.19576e6,2.19576e6,2.19576e6,0.0,1.96598e6,1.96598e6,2.19576e6,0.0,2.19576e6,2.03391e6,0.0,81324.3,1.68904e5,9.05991,35.0475,5.40555e5,5.61172e5,3.76866e5,1.66685e5,1
8,0.0,4.60684e6,4.60684e6,4.60684e6,0.0,4.36923e6,4.36923e6,4.60684e6,0.0,4.60684e6,4.43806e6,0.0,1.77186e5,3.54373e5,7.86781,17.8814,1.2066e6,1.27757e6,855112.0,3.81422e5,1
9,1.52431e7,1.54795e7,1.54795e7,1.18236e5,1.52431e7,1.52431e7,1.52431e7,2.0724e5,1.52431e7,2.36473e5,1.53113e7,1.52431e7,5.95366e5,1.19073e6,11.9209,27.895,4.22228e6,4.41639e6,2.98758e6,1.28745e6,1


In [12]:
using ScikitLearn
@sk_import neighbors:KNeighborsClassifier

PyObject <class 'sklearn.neighbors._classification.KNeighborsClassifier'>

In [13]:
# converting into vectors
K_X_train = Matrix(select(train_df, Not(:Label)))

K_y_train = Vector(train_df.Label)

K_X_test = Matrix(select(test_df, Not(:Label)))

K_y_test = Vector(test_df.Label)

111291-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [14]:
# Create an knn_model
knn_model = KNeighborsClassifier(n_neighbors=5)

PyObject KNeighborsClassifier()

In [15]:
# Fit the model to your data
ScikitLearn.fit!(knn_model, K_X_train, K_y_train)

PyObject KNeighborsClassifier()

In [16]:
# Make predictions
K_y_pred = ScikitLearn.predict(knn_model, K_X_test)


#accuracy
accuracy = mean(K_y_pred .== K_y_test)
# Calculate True Positives (TP), True Negatives (TN), False Positives (FP), and False Negatives (FN)
TP = sum((K_y_pred .== 1) .& (K_y_test .== 1))
TN = sum((K_y_pred .== 0) .& (K_y_test .== 0))
FP = sum((K_y_pred .== 1) .& (K_y_test .== 0))
FN = sum((K_y_pred .== 0) .& (K_y_test .== 1))

# Calculate precision, recall, and F1-score
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

println("Accuracy: $accuracy")
println("Precision: $precision")
println("Recall: $recall")
println("F1-Score: $f1_score")

Accuracy: 0.9114393796443558
Precision: 0.24661071659134925
Recall: 0.15277962938274897
F1-Score: 0.18867303259795853
